# Introduction

### Import the libraries

In [1]:
import pandas as pd

### Notebook display settings

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

# Data Transformation & Exploration

### Read the data

In [3]:
training_data=pd.read_csv('Data/epl-training.csv')
# Dropping the trailing columns
training_data.dropna(axis=1, how='all',inplace=True)
training_data.drop([column for column in training_data.columns if 'Unnamed' in column], axis = 1, inplace=True) 
training_data.shape

(4180, 22)

In [4]:
training_data.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,16/08/2008,Arsenal,West Brom,1,0,H,1,0,H,H Webb,24,5,14,4,11,8,7,5,0,0,0,0
1,16/08/2008,Bolton,Stoke,3,1,H,3,0,H,C Foy,14,8,8,2,13,12,4,3,1,2,0,0
2,16/08/2008,Everton,Blackburn,2,3,A,1,1,D,A Marriner,10,15,5,11,11,9,3,5,2,2,0,0
3,16/08/2008,Hull,Fulham,2,1,H,1,1,D,P Walton,11,12,6,6,10,9,5,6,3,0,0,0
4,16/08/2008,Middlesbrough,Tottenham,2,1,H,0,0,D,M Atkinson,14,8,10,5,11,12,7,9,1,2,0,0


### Exploratory Data Analysis

#### Number of matches

In [7]:
def get_pivot_column_vs_column_count(df,col1,col2):
    table=pd.DataFrame(df.groupby([col1,col2]).size()).reset_index()
    return pd.pivot_table(table,values=[0],index=col1,columns=col2)

In [6]:
get_pivot_column_vs_column_count(training_data,'HomeTeam','AwayTeam')

# Methodology Overview

# Model Training & Validation

# Results

# Final Predictions on Test Set

# Conclusion